In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.metrics.clustering import *
from ignite.metrics.regression import *
from ignite.utils import *

In [2]:
def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

In [3]:
n_inputs = 1 # number of features in input
n_output = 1 # number of features in output

# https://github.com/zalandoresearch/fashion-mnist
INPUT_DIM = 28 * 28
OUTPUT_DIM = 10
LEARNING_RATE = 0.1
NUM_EPOCHS = 1000

# -----------------------------
# 1. Generate dataset
# -----------------------------
torch.manual_seed(0)  # for reproducibility
X = torch.rand(20, INPUT_DIM)  # shape (20 samples, 1 feature)

# Binary labels: 1 if number > 0.5 else 0
Y = torch.randint(0, 2, (20, OUTPUT_DIM)).float()

# -----------------------------
# 2. Define neural network
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self, input_dim, layer_sizes, output_dim):
        super().__init__()
        layers = []

        # Input → first hidden layer
        prev = input_dim
        for size in layer_sizes:
            layers.append(nn.Linear(prev, size))
            layers.append(nn.ReLU())
            prev = size

        # Last hidden → output
        layers.append(nn.Linear(prev, output_dim))
        layers.append(nn.Sigmoid())

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# -----------------------------
# 3. Create model, loss, optimizer
# -----------------------------
model = SimpleNN(input_dim=INPUT_DIM, layer_sizes=[4, 4, 4], output_dim=OUTPUT_DIM)
loss_fn = nn.CrossEntropyLoss()               # Binary cross-entropy loss
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# -----------------------------
# 4. Training loop
# -----------------------------
for epoch in range(NUM_EPOCHS):
    # Forward pass
    y_pred = model(X)
    
    # Compute loss
    loss = loss_fn(y_pred, Y)
    
    # Backward pass and weight update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# -----------------------------
# 5. Evaluate predictions
# -----------------------------
with torch.no_grad():  # No gradient needed for evaluation
    predictions = model(X)
    predicted_classes = (predictions > 0.5).float()
    # print("\nInput numbers:")
    # print(X.T)
    # print("Predicted classes:")
    # print(predicted_classes.T)
    # print("True classes:")
    # print(Y.T)

    metric = Accuracy()
    metric.attach(default_evaluator, "accuracy")
    y_true = Y.T.contiguous()
    y_pred = predicted_classes.T.contiguous()
    state = default_evaluator.run([[y_pred, y_true]])
    print(state.metrics["accuracy"])


Epoch 0, Loss: 10.9763
Epoch 100, Loss: 10.9133
Epoch 200, Loss: 10.8590
Epoch 300, Loss: 10.7082
Epoch 400, Loss: 10.5408
Epoch 500, Loss: 10.9224
Epoch 600, Loss: 10.8909
Epoch 700, Loss: 10.8828
Epoch 800, Loss: 10.8797
Epoch 900, Loss: 10.8783
0.565
